In [10]:
from rrt_star import *
import numpy as np
import matplotlib.pyplot as plt

In [2]:
import roboticstoolbox as rtb
import spatialmath as sm # provides objects for representing transformations
from swift import Swift # lightweight browser-based simulator which comes with the toolbox
import spatialgeometry as sg # utility package for dealing with geometric objects

In [3]:
robot = rtb.models.wx200()
print(robot)
useful_config_idxs = [0,1,2,3,4]
useful_links = lambda robot: [robot.links[i] for i in useful_config_idxs]
def pad(q):
	return list(q) + [0]*3

ERobot: wx200 (by Interbotix), 8 joints (RRRRRRPP), 4 branches, dynamics, geometry, collision
┌──────┬─────────────────────┬───────┬───────────────────┬───────────────────────────────────────────┐
│ link │        link         │ joint │      parent       │            ETS: parent to link            │
├──────┼─────────────────────┼───────┼───────────────────┼───────────────────────────────────────────┤
│    0 │ /base_link          │       │ BASE              │ SE3()                                     │
│    1 │ /shoulder_link      │     0 │ /base_link        │ SE3(0, 0, 0.0716) ⊕ Rz(q0)                │
│    2 │ /upper_arm_link     │     1 │ /shoulder_link    │ SE3(0, 0, 0.03865) ⊕ Ry(q1)               │
│    3 │ /forearm_link       │     2 │ /upper_arm_link   │ SE3(0.05, 0, 0.2; 180°, -0°, 0°) ⊕ Ry(q2) │
│    4 │ /wrist_link         │     3 │ /forearm_link     │ SE3(0.2, 0, 0) ⊕ Ry(q3)                   │
│    5 │ /gripper_link       │     4 │ /wrist_link       │ SE3(0.065, 0, 0; -180°,

In [4]:
def has_pairwise_collision(q):
	robot.q = pad(q)
	links = useful_links(robot)
	n_links = len(links)
	for i1 in range(n_links):
		link1 = links[i1]
		for i2 in list(range(0,i1-1))+list(range(i1+2,n_links)):
			link2 = links[i2]
			for collider1 in link1.collision:
				if collider1.stype == 'mesh':
					continue
				for collider2 in link2.collision:
					if collider2.stype == 'mesh':
						continue
					if collider1.iscollided(collider2):
						# print(collider1, '\n with \n', collider2)
						return True

	return False

# has_pairwise_collision(robot.q[useful_config_idxs])

In [5]:
goal_ee_T = robot.fkine((1/4*robot.qlim[0]+3/4*robot.qlim[1]), end='/ee_arm_link')
goal_estimate = robot.ikine_QP(Tep=goal_ee_T, end='/ee_arm_link').q

def goal_test(q):
	ee_T = robot.fkine(pad(q), end='/ee_arm_link')
	return np.sum(np.abs(ee_T-goal_ee_T)) < 0.01

In [6]:
robot.qr

array([ 0.        , -0.3       ,  0.        , -2.2       ,  0.        ,
        2.        ,  0.78539816,  0.        ])

In [14]:
start_config = np.zeros(5) # Initial configuration

planner = RRTStar(start=start_config, goal=goal_estimate, 
					minCoords=robot.qlim[0][useful_config_idxs], maxCoords=robot.qlim[1][useful_config_idxs], 
					goal_test=goal_test, step_size=0.1, search_radius=0.4, collisionChecker=has_pairwise_collision)
path = planner.run_rrt_star()

if path:
	print("Found path:")
	for config in path:
		print(config)
else:
	print("Path not found.")

Goal reached in 562 iterations.
Found path:
[0. 0. 0. 0. 0.]
[-0.2156095   0.09819067 -0.09659831 -0.06958035 -0.28297593]
[-0.35632744  0.05607277 -0.02679422 -0.24129619 -0.50826538]
[-0.55329232  0.00805691 -0.09311917 -0.34865684 -0.60958045]
[-0.70470057 -0.08832827 -0.0630303  -0.47138385 -0.79789026]
[-0.76048162 -0.35953205  0.00785622 -0.63238877 -0.91129001]
[-0.96991673 -0.5980427   0.13139768 -0.65701516 -0.80714628]
[-1.21864615 -0.78372948  0.24053656 -0.60398049 -0.88542083]
[-1.26535556 -0.91807622  0.34618786 -0.58547067 -0.97632944]
[-1.39776059 -1.15045728  0.57475329 -0.55768791 -1.15883407]
[-1.4395247  -1.40410347  0.76601456 -0.56348679 -1.32992311]
[-1.48648407 -1.54450727  0.86297465 -0.52805217 -1.4160899 ]
[-1.57079633 -1.79659243  1.03705968 -0.4644318  -1.57079633]
[-1.57079633 -1.79659243  1.03705968 -0.4644318  -1.57079633]


In [15]:
ee_axes = sg.Axes(0.1)
goal_axes = sg.Axes(0.1)
goal_axes.T = goal_ee_T

# Make a new environment and add our robot
env = Swift()
env.launch(realtime=True, browser="notebook")
env.add(robot)
env.add(ee_axes)
env.add(goal_axes)

# Change the robot configuration to the ready position
# robot._T = np.diag(np.ones(4))
# robot.qr = np.zeros(8)
robot.q = pad(start_config[:])

# Step the sim to view the robot in this configuration
env.step(0.01)

dt = 1.0
time = 0

for step_i in range(len(path)):
	robot.q = pad(path[step_i])

	# Update the ee axes
	ee_axes.T = robot.fkine(robot.q, end='/ee_arm_link').A
	
	# Step the simulator by dt seconds
	env.step(dt)
	time += dt

In [18]:
for i in range(1, len(path)):
	print(np.linalg.norm(path[i]-path[i-1]))

0.38778460603014026
0.3266339479208582
0.2594045817544104
0.2892099673641978
0.3470878839633527
0.3570227824602149
0.3423401542593008
0.19999999999999996
0.3973084760208554
0.36327206282351926
0.19999999999999993
0.3590859395602983
0.0
